In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

from sklearn.decomposition import NMF
from scipy.sparse.linalg import svds

#set the max columns to none
pd.set_option('display.max_columns', None)

## I will perform matrix factorization to the interaction matrix

In [9]:
# Read interaction matrix pickle file
fillna_value = "0"
size = 'demo'
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{size}_{fillna_value}.pkl'
interaction_matrix = pd.read_pickle(interaction_matrix_file_path)
print(interaction_matrix.shape)

(1590, 4247)


In [10]:
interaction_matrix.head(2)

article_id_fixed  9251369  9730301  9733713  9737535  9738292  9738334  \
user_id                                                                  
11313               299.0    323.0      7.0   1038.0     32.0      4.0   
13538                 0.0      3.0     14.0      0.0      0.0      0.0   

article_id_fixed  9740021  9740161  9740174  9740356  9743748  9743795  \
user_id                                                                  
11313                82.0      4.0    674.0      9.0   1325.0     49.0   
13538                 0.0      0.0      0.0      4.0      0.0      0.0   

article_id_fixed  9743818  9743911  9744457  9745419  9745698  9745723  \
user_id                                                                  
11313               303.0     53.0    176.0     26.0     91.0     82.0   
13538                 0.0      0.0      0.0      0.0     11.0      7.0   

article_id_fixed  9745733  9745769  9747443  9748918  9749849  9750772  \
user_id                                                                  
11313               124.0      8.0      2.0     15.0     31.0    714.0   
13538                 4.0      7.0      5.0      0.0      0.0      0.0   

article_id_fixed  9753177  9753518  9753521  9753543  9753545  9753949  \
user_id                                                                  
11313               184.0     36.0      5.0     72.0    264.0      0.0   
13538                 0.0      0.0      0.0      0.0      0.0      3.0   

article_id_fixed  9753986  9754025  9754271  9754413  9754520  9755361  \
user_id                                                                  
11313                21.0     97.0    328.0    972.0      6.0     12.0   
13538                 0.0      0.0      0.0      0.0     65.0      4.0   

article_id_fixed  9755511  9755537  9755571  9755964  9757181  9757183  \
user_id                                                                  
11313                 1.0    239.0     50.0    753.0      0.0     48.0   
13538                 0.0      0.0      0.0     25.0      0.0      0.0   

article_id_fixed  9757198  9757216  9757236  9757276  9757552  9757622  \
user_id                                                                  
11313               422.0      0.0     21.0      2.0     23.0     17.0   
13538                 0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9757676  9758567  9759063  9759103  9759109  9759162  \
user_id                                                                  
11313                30.0    516.0    130.0   1138.0      5.0      8.0   
13538                 0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9759353  9760735  9761255  9761391  9761464  9761620  \
user_id                                                                  
11313                 2.0    156.0     64.0    309.0     19.0     55.0   
13538                 0.0      0.0      0.0      0.0      3.0      3.0   

article_id_fixed  9761683  9761772  9765641  9765804  9765871  9765973  \
user_id                                                                  
11313                 7.0      8.0      0.0     25.0      0.0     36.0   
13538                 0.0      0.0      6.0      0.0      0.0      0.0   

article_id_fixed  9766007  9766161  9766279  9767336  9767639  9767675  \
user_id                                                                  
11313               644.0      5.0     18.0      4.0     16.0      5.0   
13538                 0.0      6.0      9.0      0.0      0.0      0.0   

article_id_fixed  9769244  9769387  7594265  7787495  9550399  9667501  \
user_id                                                                  
11313                15.0     15.0      0.0      0.0      0.0      0.0   
13538                 0.0      0.0      5.0      1.0      3.0     76.0   

article_id_fixed  9685856  9716537  9725978  9727216  9728819  9729839  \
user_id                                                          

In [11]:
interaction_matrix.head(2)

article_id_fixed  9251369  9730301  9733713  9737535  9738292  9738334  \
user_id                                                                  
11313               299.0    323.0      7.0   1038.0     32.0      4.0   
13538                 0.0      3.0     14.0      0.0      0.0      0.0   

article_id_fixed  9740021  9740161  9740174  9740356  9743748  9743795  \
user_id                                                                  
11313                82.0      4.0    674.0      9.0   1325.0     49.0   
13538                 0.0      0.0      0.0      4.0      0.0      0.0   

article_id_fixed  9743818  9743911  9744457  9745419  9745698  9745723  \
user_id                                                                  
11313               303.0     53.0    176.0     26.0     91.0     82.0   
13538                 0.0      0.0      0.0      0.0     11.0      7.0   

article_id_fixed  9745733  9745769  9747443  9748918  9749849  9750772  \
user_id                                                                  
11313               124.0      8.0      2.0     15.0     31.0    714.0   
13538                 4.0      7.0      5.0      0.0      0.0      0.0   

article_id_fixed  9753177  9753518  9753521  9753543  9753545  9753949  \
user_id                                                                  
11313               184.0     36.0      5.0     72.0    264.0      0.0   
13538                 0.0      0.0      0.0      0.0      0.0      3.0   

article_id_fixed  9753986  9754025  9754271  9754413  9754520  9755361  \
user_id                                                                  
11313                21.0     97.0    328.0    972.0      6.0     12.0   
13538                 0.0      0.0      0.0      0.0     65.0      4.0   

article_id_fixed  9755511  9755537  9755571  9755964  9757181  9757183  \
user_id                                                                  
11313                 1.0    239.0     50.0    753.0      0.0     48.0   
13538                 0.0      0.0      0.0     25.0      0.0      0.0   

article_id_fixed  9757198  9757216  9757236  9757276  9757552  9757622  \
user_id                                                                  
11313               422.0      0.0     21.0      2.0     23.0     17.0   
13538                 0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9757676  9758567  9759063  9759103  9759109  9759162  \
user_id                                                                  
11313                30.0    516.0    130.0   1138.0      5.0      8.0   
13538                 0.0      0.0      0.0      0.0      0.0      0.0   

article_id_fixed  9759353  9760735  9761255  9761391  9761464  9761620  \
user_id                                                                  
11313                 2.0    156.0     64.0    309.0     19.0     55.0   
13538                 0.0      0.0      0.0      0.0      3.0      3.0   

article_id_fixed  9761683  9761772  9765641  9765804  9765871  9765973  \
user_id                                                                  
11313                 7.0      8.0      0.0     25.0      0.0     36.0   
13538                 0.0      0.0      6.0      0.0      0.0      0.0   

article_id_fixed  9766007  9766161  9766279  9767336  9767639  9767675  \
user_id                                                                  
11313               644.0      5.0     18.0      4.0     16.0      5.0   
13538                 0.0      6.0      9.0      0.0      0.0      0.0   

article_id_fixed  9769244  9769387  7594265  7787495  9550399  9667501  \
user_id                                                                  
11313                15.0     15.0      0.0      0.0      0.0      0.0   
13538                 0.0      0.0      5.0      1.0      3.0     76.0   

article_id_fixed  9685856  9716537  9725978  9727216  9728819  9729839  \
user_id                                                          

In [12]:
# ---------(NOT USED)-----------------
# # Apply matrix factorization 
# nmf = NMF(n_components=5)
# user_matrix = nmf.fit_transform(interaction_matrix)
# article_matrix = nmf.components_.T


# Perform Singular Value Decomposition (SVD) on the user-article interaction matrix
k = 300  # Number of latent factors (embedding dimensions)

U, sigma, Vt = svds(interaction_matrix.values, k=k)  # Perform SVD, obtaining U, sigma, Vt

# Diagonalize the singular values sigma to create a diagonal matrix
sigma_diag = np.diag(sigma)

# Reconstruct the user and article embeddings
user_matrix = np.dot(U, np.sqrt(sigma_diag))  # User embeddings
article_matrix = np.dot(np.sqrt(sigma_diag), Vt).T  # Article embeddings

In [17]:
print(user_matrix.shape)
print(article_matrix.shape)

(1590, 300)
(4247, 300)


In [18]:
column_lst = interaction_matrix.columns.tolist()

user_matrix_df = pd.DataFrame(user_matrix, index=interaction_matrix.index)
article_matrix_df = pd.DataFrame(article_matrix, index=column_lst)

In [19]:
user_matrix_df.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11313,-2.825051,-0.367806,0.030133,-2.053737,3.554389,1.388510,-2.562073,-1.092667,-2.818263,2.857499,0.831981,1.205728,6.435611,4.833850,-1.423295,1.175506,-3.031063,5.992809,1.439074,1.426760,4.614215,-1.596846,-0.986633,6.872793,-4.591221,-2.776274,8.798182,4.116849,1.265229,-7.052317,10.766291,-2.082953,-7.113878,-7.102123,4.990719,5.882839,2.997373,0.434430,0.596505,-0.965060,2.409810,1.690596,3.834605,-2.174452,0.592399,5.877130,6.036171,1.514004,-3.373987,4.354969,2.718636,-1.081442,6.151529,0.235764,4.191148,3.761474,-5.128614,1.487922,1.149899,0.917394,-0.924723,-1.182993,-0.963209,-7.559617,0.178489,0.529260,-2.731196,-1.814501,-5.120493,-2.550373,-3.007591,1.165470,-0.488246,-0.740266,-1.309792,-0.570396,-7.118636,4.378279,-0.780332,-0.038405,-4.739377,1.151919,-1.608566,3.602339,1.232579,-2.577470,-4.218756,5.953595,-4.026121,-2.906991,2.879662,-1.312774,-1.929675,1.813092,-3.101846,-0.140112,2.028189,-3.616562,-1.360866,0.593890,5.85206,-3.017415,-0.932006,2.909566,3.425526,0.694219,0.439433,-2.612690,-1.439188,0.838306,-3.876763,-1.919920,-2.208586,-4.637822,0.143225,3.001009,1.771870,-0.413992,-2.630573,-1.096379,-2.228485,-1.912777,1.113632,-2.241936,-0.916456,1.076158,0.815738,-4.954364,-0.025852,-0.009388,-1.347658,0.374764,-4.638402,-0.130800,1.001182,1.892513,2.190351,-0.266270,-3.705693,-2.498474,1.184676,-2.024988,1.478558,0.627085,-2.929154,1.940609,1.832793,-0.880001,2.627166,-1.489843,2.296835,0.877987,-0.742256,2.194758,0.618173,0.424943,2.298580,0.756355,-0.194605,1.327212,-0.877367,-3.340565,2.345934,2.298056,-0.829447,2.078891,-3.723301,-3.054250,-1.053197,-0.779701,-0.282118,2.558045,0.374072,-1.016986,-0.508164,0.757217,0.088140,-3.313303,-0.183975,-1.820544,-3.985510,-1.139981,-0.593812,-0.609611,1.526351,0.612285,0.669950,-0.083153,1.318586,0.329908,-3.014675,0.544131,0.322205,-2.302082,-1.666159,-0.727223,1.278348,0.759147,-0.277301,-0.696879,-0.294723,-1.805701,-0.699109,-1.967577,-0.800884,0.682890,-1.763434,-0.663419,1.438010,-0.831826,2.173392,1.199717,1.082985,-2.159236,-0.379038,0.453479,0.617694,0.767890,-0.243861,-1.071472,0.101969,-0.247214,-1.692061,-0.001510,-0.966586,-1.319573,2.291144,0.692360,0.057022,-0.418317,1.985151,-3.056859,-0.277265,1.400336,0.083405,0.796284,-1.267823,-0.568115,-1.219499,0.431303,0.524081,0.218642,-0.028769,0.503232,1.252511,1.538461,-1.139012,2.279868,1.249102,0.555536,2.366194,-0.828976,0.652409,1.483474,-1.097733,1.939587,1.894045,-0.849952,-0.982699,-0.378268,-0.026221,-1.497036,0.482921,0.536055,2.117766,1.137666,0.240610,-2.480329,-0.781504,0.491179,0.512972,0.477186,1.175084,-

In [20]:
article_matrix_df.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
9251369,-0.30518,-0.039914,0.000081,-0.219868,0.370485,0.148578,-0.270606,-0.111830,-0.298030,0.296399,0.088596,0.127368,0.666595,0.495312,-0.147069,0.115975,-0.309882,0.620912,0.143805,0.143093,0.469781,-0.163797,-0.105983,0.690247,-0.465840,-0.277042,0.882007,0.410240,0.126284,-0.694077,1.074132,-0.206281,-0.702260,-0.701202,0.493876,0.576642,0.291921,0.040876,0.053708,-0.091893,0.230951,0.166469,0.370222,-0.209084,0.061173,0.568357,0.575684,0.147137,-0.321696,0.414822,0.256593,-0.106136,0.582886,0.018501,0.398085,0.353696,-0.481472,0.142180,0.107665,0.086215,-0.082668,-0.106758,-0.090319,-0.695190,0.014249,0.049716,-0.248648,-0.169884,-0.463284,-0.233658,-0.271040,0.108446,-0.044363,-0.064954,-0.118204,-0.049588,-0.635291,0.388937,-0.063387,-0.003056,-0.411033,0.100812,-0.140257,0.317469,0.111648,-0.224160,-0.365058,0.511841,-0.344885,-0.248950,0.248576,-0.111440,-0.166613,0.158108,-0.259717,-0.014349,0.177165,-0.307822,-0.110053,0.048608,0.481812,-0.250023,-0.081069,0.240127,0.281473,0.056012,0.037836,-0.208130,-0.115056,0.075567,-0.311959,-0.154512,-0.171986,-0.381250,0.004645,0.240222,0.142972,-0.033379,-0.215080,-0.086611,-0.175668,-0.154745,0.083649,-0.179273,-0.081983,0.082983,0.062194,-0.383718,0.001802,0.002069,-0.101744,0.030065,-0.357333,-0.005693,0.085092,0.147302,0.169745,-0.017653,-0.278312,-0.184664,0.090371,-0.159351,0.106627,0.043147,-0.214680,0.141470,0.133250,-0.060492,0.192193,-0.105287,0.163659,0.056418,-0.053725,0.154414,0.048216,0.031932,0.172448,0.057249,-0.012363,0.097976,-0.059147,-0.228245,0.165827,0.158520,-0.057508,0.148897,-0.257258,-0.207079,-0.069532,-0.046090,-0.016534,0.170827,0.027675,-0.070086,-0.033323,0.049352,0.005263,-0.217766,-0.011882,-0.117169,-0.262363,-0.069629,-0.042360,-0.037700,0.098303,0.046035,0.048157,-0.005569,0.084066,0.020900,-0.193481,0.031462,0.023777,-0.150106,-0.103623,-0.037255,0.070632,0.050466,-0.022131,-0.033231,-0.013885,-0.108925,-0.031461,-0.122502,-0.052188,0.035517,-0.091474,-0.031974,0.084370,-0.046469,0.133844,0.063618,0.053038,-0.125770,-0.014522,0.028377,0.026508,0.051333,-0.010965,-0.061514,0.003704,-0.017954,-0.099232,0.000121,-0.054310,-0.066685,0.135349,0.039316,0.000122,-0.014551,0.107390,-0.160972,-0.020793,0.087389,0.012276,0.034679,-0.064340,-0.028938,-0.071006,0.009715,0.025218,0.000251,-0.002369,0.019860,0.059038,0.078925,-0.051878,0.112603,0.057798,0.024986,0.118204,-0.031820,0.031629,0.083337,-0.054656,0.075553,0.083315,-0.038463,-0.046738,-0.022848,-0.000651,-0.073601,0.030826,0.032962,0.090550,0.048962,0.007214,-0.105175,-0.025681,0.002408,0.022533,0.017650,0.045018,-0.041706,0.060468,-0.046811,0.008508,-0.065954,-0.034171,0.008039,-0.057653,0.027919,0.038367,-0.020610,0.055413,-0.036190,0.008002,0.064833,0.011713,-0.071833,0.214525,0.005896,0.017027,-0.037189,0.014235,0.017839,0.030130,-0.044657,-0.001425,0.000168
9730301,-0.03106,0.440268,0.364339,0.325598,-0.209704,-0.

In [21]:
# Save the cosine similarity matrix to pickle file cause is very big

user_matrix_df_file_path = f'./files/pickle/user_matrix_{size}_{fillna_value}.pkl'
article_matrix_df_file_path = f'./files/pickle/article_matrix_{size}_{fillna_value}.pkl'

user_matrix_df.to_pickle(user_matrix_df_file_path)
article_matrix_df.to_pickle(article_matrix_df_file_path)

## Other

In [5]:
# # Define a function to convert strings back to lists
# def str_to_list(s):
#     try:
#         temp_lst = s.replace('[', '').replace(']','').replace(' ', '').split(',')
        
#         return [float(x) for x in temp_lst]
#     except:
#         return s


# for col in tqdm(interaction_matrix.columns.to_list()):
#     interaction_matrix[col] = interaction_matrix[col].apply(str_to_list)

/var/folders/ld/c07vs56s18bc0h2krlvl0myr0000gn/T/ipykernel_2337/1990072283.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for col in tqdm(interaction_matrix.columns.to_list()):


  0%|          | 0/4248 [00:00<?, ?it/s]